<a href="https://colab.research.google.com/github/Gutomoby/BolsaBrasileira/blob/main/Dados_da_CVM_(DFP)_dos_Balan%C3%A7os_Empresas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Baixando dados dos Balanços das Empresas Listadas em Bolsa

### Base de Dados CVM

http://dados.cvm.gov.br

# Configurações Iniciais

In [ ]:
!pip install wget

  Created wheel for wget: filename=wget-3.2-cp36-none-any.whl size=9682 sha256=4f02d76f7ad837a382da84eb74b3dc9634efea3b67e8324715cfcf2dffe8eca2
  Stored in directory: /root/.cache/pip/wheels/40/15/30/7d8f7cea2902b4db79e3fea550d7d7b85ecb27ef992b618f3f
Successfully built wget


In [ ]:
import pandas as pd
import wget
from zipfile import ZipFile
from google.colab.data_table import DataTable
import plotly.graph_objects as go

## Acessando a base de dados e criando arquivos históricos

In [ ]:
url_base = 'http://dados.cvm.gov.br/dados/CIA_ABERTA/DOC/DFP/DADOS/'

* criando uma lista com o nome de todos os arquivos

In [ ]:
arquivos_zip = []
for ano in range(2010,2021):
  arquivos_zip.append(f'dfp_cia_aberta_{ano}.zip')

arquivos_zip

['dfp_cia_aberta_2010.zip',
 'dfp_cia_aberta_2011.zip',
 'dfp_cia_aberta_2012.zip',
 'dfp_cia_aberta_2013.zip',
 'dfp_cia_aberta_2014.zip',
 'dfp_cia_aberta_2015.zip',
 'dfp_cia_aberta_2016.zip',
 'dfp_cia_aberta_2017.zip',
 'dfp_cia_aberta_2018.zip',
 'dfp_cia_aberta_2019.zip',
 'dfp_cia_aberta_2020.zip']

* fazendo o download de todos os arquivos

In [ ]:
for arq in arquivos_zip:
  wget.download(url_base+arq)

* extraindo os arquivos zip

In [ ]:
for arq in arquivos_zip:
  ZipFile(arq, 'r').extractall('CVM')

* criandos arquivos históricos

In [ ]:
!mkdir DADOS

In [ ]:
nomes = ['BPA_con', 'BPA_ind', 'BPP_con', 'BPP_ind', 'DFC_MD_con', 'DFC_MD_ind', 'DFC_MI_con', 'DFC_MI_ind', 'DMPL_con', 'DMPL_ind', 'DRE_con', 'DRE_ind', 'DVA_con', 'DVA_ind']
for nome in nomes:
  arquivo = pd.DataFrame()
  for ano in range(2010,2021):
    arquivo = pd.concat([arquivo, pd.read_csv(f'CVM/dfp_cia_aberta_{nome}_{ano}.csv', sep=';', decimal=',', encoding='ISO-8859-1')])
  arquivo.to_csv(f'DADOS/dfp_cia_aberta_{nome}_2010-2020.csv', index=False)


In [ ]:
dre = pd.read_csv('/content/DADOS/dfp_cia_aberta_DRE_ind_2010-2020.csv')

In [ ]:
dre = dre[dre['ORDEM_EXERC'] == "ÚLTIMO"]

In [ ]:
dre.head(3)

,CNPJ_CIA,DT_REFER,VERSAO,DENOM_CIA,CD_CVM,GRUPO_DFP,MOEDA,ESCALA_MOEDA,ORDEM_EXERC,DT_INI_EXERC,DT_FIM_EXERC,CD_CONTA,DS_CONTA,VL_CONTA,ST_CONTA_FIXA
1,00.000.000/0001-91,2010-12-31,3,BCO BRASIL S.A.,1023,DF Individual - Demonstração do Resultado,REAL,MIL,ÚLTIMO,2010-01-01,2010-12-31,3.01,Receitas da Intermediação Financeira,72173904.0,S
3,00.000.000/0001-91,2010-12-31,3,BCO BRASIL S.A.,1023,DF Individual - Demonstração do Resultado,REAL,MIL,ÚLTIMO,2010-01-01,2010-12-31,3.01.01,Operações de Crédito,47391099.0,N
5,00.000.000/0001-91,2010-12-31,3,BCO BRASIL S.A.,1023,DF Individual - Demonstração do Resultado,REAL,MIL,ÚLTIMO,2010-01-01,2010-12-31,3.01.02,Operações de Arrendamento Mercantil,40372.0,N


In [ ]:
empresas = dre[['DENOM_CIA','CD_CVM']].drop_duplicates().set_index('CD_CVM')
DataTable(empresas)

,DENOM_CIA
CD_CVM,
1023,BCO BRASIL S.A.
14206,BRB BCO DE BRASILIA S.A.
2437,CENTRAIS ELET BRAS S.A. - ELETROBRAS
14451,CIA ENERGETICA DE BRASILIA
14540,VBC ENERGIA SA
...,...
24970,BASE SECURITIZADORA DE CRÉDITOS IMOBILIÁRIOS S.A.
25054,CONSIG 6 I COMPANHIA SECURITIZADORA DE CRÉDITO...
24953,"ALLPARK EMPREENDIMENTOS, PARTICIPAÇÕES E SERVI..."


In [ ]:
empresa = dre[dre['CD_CVM'] == 7617]

In [ ]:
empresa.head()

,CNPJ_CIA,DT_REFER,VERSAO,DENOM_CIA,CD_CVM,GRUPO_DFP,MOEDA,ESCALA_MOEDA,ORDEM_EXERC,DT_INI_EXERC,DT_FIM_EXERC,CD_CONTA,DS_CONTA,VL_CONTA,ST_CONTA_FIXA
33808,61.532.644/0001-15,2010-12-31,1,ITAUSA S.A.,7617,DF Individual - Demonstração do Resultado,REAL,MIL,ÚLTIMO,2010-01-01,2010-12-31,3.01,Receita de Venda de Bens e/ou Serviços,0.0,S
33810,61.532.644/0001-15,2010-12-31,1,ITAUSA S.A.,7617,DF Individual - Demonstração do Resultado,REAL,MIL,ÚLTIMO,2010-01-01,2010-12-31,3.02,Custo dos Bens e/ou Serviços Vendidos,0.0,S
33812,61.532.644/0001-15,2010-12-31,1,ITAUSA S.A.,7617,DF Individual - Demonstração do Resultado,REAL,MIL,ÚLTIMO,2010-01-01,2010-12-31,3.03,Resultado Bruto,0.0,S
33814,61.532.644/0001-15,2010-12-31,1,ITAUSA S.A.,7617,DF Individual - Demonstração do Resultado,REAL,MIL,ÚLTIMO,2010-01-01,2010-12-31,3.04,Despesas/Receitas Operacionais,4404000.0,S
33816,61.532.644/0001-15,2010-12-31,1,ITAUSA S.A.,7617,DF Individual - Demonstração do Resultado,REAL,MIL,ÚLTIMO,2010-01-01,2010-12-31,3.04.01,Despesas com Vendas,0.0,S


In [ ]:
DataTable(empresa[['CD_CONTA', 'DS_CONTA']].drop_duplicates().set_index('CD_CONTA'))

,DS_CONTA
CD_CONTA,
3.01,Receita de Venda de Bens e/ou Serviços
3.02,Custo dos Bens e/ou Serviços Vendidos
3.03,Resultado Bruto
3.04,Despesas/Receitas Operacionais
3.04.01,Despesas com Vendas
3.04.02,Despesas Gerais e Administrativas
3.04.03,Perdas pela Não Recuperabilidade de Ativos
3.04.04,Outras Receitas Operacionais
3.04.05,Outras Despesas Operacionais


In [ ]:
conta = empresa[empresa['CD_CONTA'] == '3.99.01.02']

In [ ]:
conta.index = pd.to_datetime(conta['DT_REFER'])
conta.head()

,CNPJ_CIA,DT_REFER,VERSAO,DENOM_CIA,CD_CVM,GRUPO_DFP,MOEDA,ESCALA_MOEDA,ORDEM_EXERC,DT_INI_EXERC,DT_FIM_EXERC,CD_CONTA,DS_CONTA,VL_CONTA,ST_CONTA_FIXA
DT_REFER,,,,,,,,,,,,,,,
2010-12-31,61.532.644/0001-15,2010-12-31,1,ITAUSA S.A.,7617,DF Individual - Demonstração do Resultado,REAL,MIL,ÚLTIMO,2010-01-01,2010-12-31,3.99.01.02,PN,1.01,N
2011-12-31,61.532.644/0001-15,2011-12-31,1,ITAUSA S.A.,7617,DF Individual - Demonstração do Resultado,REAL,MIL,ÚLTIMO,2011-01-01,2011-12-31,3.99.01.02,PN,1.10,N
2012-12-31,61.532.644/0001-15,2012-12-31,1,ITAUSA S.A.,7617,DF Individual - Demonstração do Resultado,REAL,MIL,ÚLTIMO,2012-01-01,2012-12-31,3.99.01.02,PN,0.94,N
2013-12-31,61.532.644/0001-15,2013-12-31,1,ITAUSA S.A.,7617,DF Individual - Demonstração do Resultado,REAL,MIL,ÚLTIMO,2013-01-01,2013-12-31,3.99.01.02,PN,1.05,N
2014-12-31,61.532.644/0001-15,2014-12-31,1,ITAUSA S.A.,7617,DF Individual - Demonstração do Resultado,REAL,MIL,ÚLTIMO,2014-01-01,2014-12-31,3.99.01.02,PN,1.30,N


# Calculando PL

In [ ]:
!pip install yfinance

In [ ]:
import yfinance as yf

In [ ]:
prices = yf.download('ITSA4.SA', start='2011-01-01')[['Adj Close', 'Close']]

[*********************100%***********************]  1 of 1 completed


In [ ]:
prices.head()

,Adj Close,Close
Date,,
2011-01-03,4.333224,7.28339
2011-01-04,4.388986,7.37712
2011-01-05,4.444757,7.47086
2011-01-06,4.349625,7.31096
2011-01-07,4.221695,7.09593


In [ ]:
indicadores = prices.join(conta['VL_CONTA'], how='outer')

In [ ]:
indicadores.rename({'VL_CONTA':'LPA'}, axis=1, inplace=True)

In [ ]:
indicadores.fillna(method='ffill', inplace=True)

In [ ]:
indicadores.dropna(inplace=True)

In [ ]:
indicadores

,Adj Close,Close,LPA
2011-01-03,4.333224,7.28339,1.01
2011-01-04,4.388986,7.37712,1.01
2011-01-05,4.444757,7.47086,1.01
2011-01-06,4.349625,7.31096,1.01
2011-01-07,4.221695,7.09593,1.01
...,...,...,...
2020-10-07,8.900000,8.90000,1.23
2020-10-08,9.330000,9.33000,1.23
2020-10-09,9.360000,9.36000,1.23
2020-10-13,9.320000,9.32000,1.23


In [ ]:
indicadores['PL'] = indicadores['Close'] / indicadores['LPA']
indicadores['PL_ajustado'] = indicadores['Adj Close'] / indicadores['LPA']

In [ ]:
indicadores

,Adj Close,Close,LPA,PL,PL_ajustado
2011-01-03,4.333224,7.28339,1.01,7.211277,4.290321
2011-01-04,4.388986,7.37712,1.01,7.304079,4.345531
2011-01-05,4.444757,7.47086,1.01,7.396891,4.400749
2011-01-06,4.349625,7.31096,1.01,7.238574,4.306560
2011-01-07,4.221695,7.09593,1.01,7.025673,4.179896
...,...,...,...,...,...
2020-10-07,8.900000,8.90000,1.23,7.235772,7.235772
2020-10-08,9.330000,9.33000,1.23,7.585366,7.585366
2020-10-09,9.360000,9.36000,1.23,7.609756,7.609756
2020-10-13,9.320000,9.32000,1.23,7.577236,7.577236


In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=indicadores.index, y=indicadores['PL'], name='PL'))
fig.add_trace(go.Scatter(x=indicadores.index, y=indicadores['PL_ajustado'], name='PL_ajustado'))
fig.add_trace(go.Scatter(x=indicadores.index, y=indicadores['Close'], name='ITSA4'))
fig.add_trace(go.Scatter(x=indicadores.index, y=indicadores['Adj Close'], name='ITSA4_Ajustado'))